In [1]:
import pandas as pd
from konlpy.tag import Okt

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [5]:
okt = Okt()

In [2]:
positive_dict = pd.read_csv('positive_score_dict.csv')
positive_dict = positive_dict.drop_duplicates(subset='단어')
positive_dict.set_index('단어', inplace=True)

result2 = pd.read_csv('community_token_final.csv', low_memory=False)

In [4]:
# 각 열에 대해 okt.morphs(stem=True) 적용 및 문자열 변환

result2['형용사_어간'] = result2['내용_형용사_str'].apply(
lambda x: ', '.join(okt.morphs(x, stem=True)) if isinstance(x, str) and x != 'NaN' else ''
)

result2['부사_어간'] = result2['내용_부사_str'].apply(
lambda x: ', '.join(okt.morphs(x, stem=True)) if isinstance(x, str) and x != 'NaN' else ''
)

result2['동사_어간'] = result2['내용_동사_str'].apply(
lambda x: ', '.join(okt.morphs(x, stem=True)) if isinstance(x, str) and x != 'NaN' else ''
)

In [7]:
result2.head(1)

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str,형용사_어간,부사_어간,동사_어간,긍정점수
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다","둥글다, ,, 있다, ,, 동일하다, ,, 있다",,"되어다, ,, 오다, ,, 보이다",-1


In [ ]:
# 행별 총 긍정 점수를 계산할 파생열 추가
row_scores = [0] * len(result2)  # 초기화

for idx in range(len(result2)):  # 데이터프레임 행 반복
    row_score = 0  # 행별 총 점수 초기화

    for column in ['내용_명사_str', '형용사_어간', '부사_어간', '동사_어간']:
        word_list = result2.loc[idx, column]  # 각 열의 데이터 가져오기
        print(f"\n행 {idx}, 열 {column} 처리 중: {word_list}")
        
        if pd.isna(word_list) or word_list.strip() == "":  # Null 또는 빈 문자열 처리
            print(f"행 {idx}, 열 {column}은 결측값 또는 빈 문자열입니다. 건너뜁니다.")
            continue
        
        words = word_list.split(', ')  # 쉼표와 공백으로 구분
        for word in words:
            word = word.strip()  # 공백 제거
            if word in positive_dict.index:
                score = positive_dict.loc[word, '점수']
                row_score += score  # 점수 합산
                print(f"  단어: {word}, 점수: {score}, 누적 점수: {row_score}")
            else:
                print(f"  단어: {word}는 사전에 없음")

    row_scores[idx] = row_score  # 행별 총 점수 저장
    print(f"행 {idx}의 총 점수: {row_score}")

# 점수 변환: 음수는 -1, 0은 0, 양수는 1
try:
    result2['긍정점수'] = [1 if score > 0 else 0 if score < 0 else -1 for score in row_scores]
except Exception as e:
    print(f"점수 변환 중 에러 발생: {e}")
    print(f"row_scores 내용: {row_scores}")
    print(f"tmp 길이: {len(result2)}, row_scores 길이: {len(row_scores)}")

# 최종 점수 출력
print("\n행별 긍정점수:")
print(result2[['긍정점수']])

In [11]:
result2['긍정점수'].value_counts()

긍정점수
-1    52095
 0    15583
 1    13394
Name: count, dtype: int64

In [23]:
result2.head(1)

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str,형용사_어간,부사_어간,동사_어간,긍정점수
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다","둥글다, ,, 있다, ,, 동일하다, ,, 있다",,"되어다, ,, 오다, ,, 보이다",1


In [50]:
# 내용 토큰화 

result2['토큰_어간'] = (
    result2['내용_명사_str'] + ', ' + 
    result2['형용사_어간'] + ', ' + 
    result2['부사_어간'] + ', ' + 
    result2['동사_어간']
)

In [52]:
result2['토큰_어간'] = result2['토큰_어간'].str.replace(r',\s*,+', ',', regex=True)  # 연속된 쉼표 제거

In [ ]:
result2.head(5)['토큰_어간'].values

In [ ]:
result2.info()

In [58]:
result2['토큰_어간'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 81072 entries, 0 to 81071
Series name: 토큰_어간
Non-Null Count  Dtype 
--------------  ----- 
71966 non-null  object
dtypes: object(1)
memory usage: 633.5+ KB


In [54]:
result2['긍정점수'].value_counts()

긍정점수
-1    52095
 0    15583
 1    13394
Name: count, dtype: int64

In [60]:
# '토큰_어간'이 NaN인 행 제거
result3 = result2.dropna(subset=['토큰_어간'])

In [61]:
result3['긍정점수'].value_counts()

긍정점수
-1    44283
 0    14858
 1    12825
Name: count, dtype: int64

In [65]:
# 언더 샘플링

sample_data_positive = result3[result3['긍정점수'] == 1].sample(12825)
sample_data_negative = result3[result3['긍정점수'] == 0].sample(12825)

total_data = pd.concat([sample_data_positive,sample_data_negative])

In [66]:
# 학습 데이터와, 테스트 데이터를 구분

train_data, test_data = train_test_split(total_data, test_size=0.25, random_state=2025)

In [67]:
# TF-IDF 변환

vectorizer = TfidfVectorizer()
# 학습 데이터 벡터화

X_train = vectorizer.fit_transform(train_data['토큰_어간'])
y_train = train_data['긍정점수'].values
# 테스트 데이터 벡터화

X_test = vectorizer.transform(test_data['토큰_어간'])
y_test = test_data['긍정점수'].values

In [68]:
# 모델 구축

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))  # 추가된 레이어
model.add(Dropout(0.5))  # 드롭아웃 추가
model.add(Dense(1, activation='sigmoid'))  # 이진 분류

d:\kdt\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [69]:
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 조기 종료 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping])  # 20%를 검증 데이터로 사용

Epoch 1/20
481/481 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7953 - loss: 0.4540 - val_accuracy: 0.9680 - val_loss: 0.0901
Epoch 2/20
481/481 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9885 - loss: 0.0410 - val_accuracy: 0.9592 - val_loss: 0.1018
Epoch 3/20
481/481 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9971 - loss: 0.0122 - val_accuracy: 0.9683 - val_loss: 0.0982
Epoch 4/20
481/481 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9993 - loss: 0.0048 - val_accuracy: 0.9717 - val_loss: 0.1010


In [70]:
# 모델 평가

y_pred = model.predict(X_test)
y_pred_classes = [1 if p > 0.5 else 0 for p in y_pred]

201/201 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [71]:
# 성능 평가

# classification_report :  정밀도(precision), 재현율(recall), F1 점수(F1 score) 등의 다양한 지표를 포함
print(classification_report(y_test, y_pred_classes))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97      3284
           1       0.96      0.97      0.96      3129

    accuracy                           0.97      6413
   macro avg       0.97      0.97      0.97      6413
weighted avg       0.97      0.97      0.97      6413



In [72]:
# 주식 커뮤니티 샘플 문장
test_samples = [
    "이 회사의 성장 가능성이 정말 기대돼요! 앞으로 주가가 오를 것 같습니다.",  # 긍정
    "이번 분기 실적이 예상보다 좋았어요. 강력한 매수 추천합니다!",  # 긍정
    "이 주식은 장기 투자로 정말 좋은 선택입니다. 믿고 투자하세요!",  # 긍정
    "이 회사는 실적이 계속 나빠지고 있어요. 이제는 매도해야 할 때인 것 같습니다.",  # 부정
    "주가가 너무 비싸요. 조정이 올 것 같아서 걱정입니다."  # 부정
]

In [73]:
X_test_samples = vectorizer.transform(test_samples)

In [74]:
# 예측
y_pred = model.predict(X_test_samples)
y_pred_classes = [1 if p > 0.5 else 0 for p in y_pred]

# 결과 출력
for sample, pred in zip(test_samples, y_pred_classes):
    sentiment = "긍정" if pred == 1 else "부정"
    print(f"문장: '{sample}' -> 예측: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
문장: '이 회사의 성장 가능성이 정말 기대돼요! 앞으로 주가가 오를 것 같습니다.' -> 예측: 긍정
문장: '이번 분기 실적이 예상보다 좋았어요. 강력한 매수 추천합니다!' -> 예측: 긍정
문장: '이 주식은 장기 투자로 정말 좋은 선택입니다. 믿고 투자하세요!' -> 예측: 긍정
문장: '이 회사는 실적이 계속 나빠지고 있어요. 이제는 매도해야 할 때인 것 같습니다.' -> 예측: 부정
문장: '주가가 너무 비싸요. 조정이 올 것 같아서 걱정입니다.' -> 예측: 부정
